after german,france and US examples.

In [ ]:
# imports
import pandas as pd
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, EasterMonday, Easter
from pandas.tseries.offsets import Day, CDay #, CustomBusinessDay
from datetime import date

You can use observance instead of offset when making the Holiday and have it return None on odd years:<br>
see > https://stackoverflow.com/questions/34708626/pandas-holiday-calendar-rule-for-us-election-day

In [ ]:
# adjusted for deciding if the 5th of May is holiday
def bevrijding_observance(dt):
    if dt.year > 2011:
        if dt.year % 5 != 0: # % 2 == 1 
            return None
        else:
            return dt
    else:
        return dt

----------------------------

more observance rules for koninginnedag, if year is before 2014, then the holiday is on 30 april<br> 
and for koningsdag, if year after 2014, then the holiday is on 27 april.

In [ ]:
def konings_observance(dt):
    if dt.year > 2013:
        return dt
    else:
        return None

def koninginne_observance(dt):
    if dt.year < 2014:
        return dt
    else:
        return None

see also > https://stackoverflow.com/questions/45654473/holiday-calendar-in-pandas-dataframe<br>
and > https://www.wikiod.com/w/Pandas_Holiday_Calendars

Dutch<br>
Hemelvaart is 39 dagen na Eerste Paasdag.<br>
Pinksteren is de zevende zondag na Pasen, dat is dus 49 dagen.<br>
Tweede Pinksterdag is dus 50 dagen na Pasen.

In [ ]:
# Pinksteren is altijd 7 weken na Pasen
# dus Pinkster-zondag = 49 dagen na pasen en Pinkster-maandag dus 50
class NLFeestdagen(AbstractHolidayCalendar):
    rules = [Holiday('Nieuwjaars Dag', month=1, day=1),
             #EasterMonday, # gives an english name
             Holiday('Paas maandag',month=1, day=1, offset=[Easter(), Day(1)]),
             Holiday('Koningsdag', month=4,day=27, observance=konings_observance),
             Holiday('Koninginnedag', month=4,day=30, observance=koninginne_observance),
             Holiday('Bevrijdingsdag', month=5, day=5, observance=bevrijding_observance),
             Holiday('Hemelvaartsdag', month=1, day=1, offset=[Easter(), Day(39)]),             
             Holiday('2e-Pinksterdag', month=1, day=1, offset=[Easter(), Day(50)]),             
             Holiday('Kerstmis',month=12,day=25),
             Holiday('2e-Kerstdag',month=12,day=26),
            ]

In [ ]:
cal = NLFeestdagen()

see > https://stackoverflow.com/questions/45654473/holiday-calendar-in-pandas-dataframe

In [ ]:
X = pd.DataFrame({'Time (CET)': pd.date_range(start='2013-01-01', end='2014-12-31', freq='24H')})

In [ ]:
holidays = cal.holidays(start=X['Time (CET)'].min(), end = X['Time (CET)'].max())

In [ ]:
# if needed to fix dtype of 'Time (CET)'
#X['Time (CET)'] = pd.to_datetime(X['Time (CET)'], dayfirst=True, errors='coerce')

In [ ]:
# pu a 1 or 0 in column 'Holiday' if date is a holiday or not
X = X.assign(Holiday=X['Time (CET)'].isin(cal.holidays()).astype(int))

In [ ]:
# show X
X

In [ ]:
# filter X op Holiday == 1
filt = (X['Holiday'] == 1 )
result = X.loc[filt]

In [ ]:
result

In [ ]:
# with holiday names
# Creating some boundaries
year = 2013 # try 2014 or later , to see change in happydays
start = date(year, 1, 1)
end = start + pd.offsets.MonthEnd(12)
happydays = cal.holidays(start=start, end=end, return_name=True)

In [ ]:
happydays

In [ ]:
happydays.reset_index(name='feestdag').rename(columns={'index':'date'})

In [ ]:
df3 = pd.DataFrame(happydays)

In [ ]:
df3.columns= ['feestdag']
df3.head()

-----------------------------------------------------------------

second example

In [ ]:
# version 2 , create dataframe with datetime column ,1100 rows 
df = pd.DataFrame({'Date':pd.date_range('2020-01-01 00:00:00', freq='24H', periods=1099)})

In [ ]:
X2 = pd.DataFrame({'Time (CET)': pd.date_range(start='2020-01-01', end='2022-12-31', freq='24H')})
holidays2 = cal.holidays(start=X2['Time (CET)'].min(), end = X2['Time (CET)'].max())

In [ ]:
# en filter op holidays
df.loc[df.Date.dt.normalize().isin(holidays2)]

---------------------------

counting workdays

In [ ]:
# Creating some boundaries
year = 2022
start = date(year, 1, 1)
end = start + pd.offsets.MonthEnd(12)

Creating a series of dates between the boundaries<br>
by using the custom calendar

In [ ]:
se = pd.bdate_range(start=start, end=end, freq=CDay(calendar=cal)).to_series()

Counting the number of working days by month

In [ ]:
se.groupby(se.dt.month).count() #.head(10)